# IFT3700 - Science des donnees - Travail 1

## Travail présenté à M. Arnaud L'Heureux et M. Alain Tapp

## Auteurs :

### Ludovic Tuncay (20174139)

### Sobhan Mohammadpour (20166971)

### Thomas Lavend'homme (20165141)

### Marc-Antoine Dufresne Gagnon (20019871)

In [0]:
!pip3 install https://github.com/scikit-learn-contrib/scikit-learn-extra/archive/master.zip
!pip3 install livelossplot > /dev/null
!pip3 install scikit-bio

import pickle
import numpy as np
import keras.backend as K
from keras.layers import *
from tqdm.auto import tqdm
from itertools import count
from keras.callbacks import *
from keras.models import Model
import matplotlib.pyplot as plt
import imgaug.augmenters as iaa
from keras.datasets import mnist
from sklearn.manifold import Isomap
from sklearn.decomposition import PCA
from keras.optimizers import SGD, Adam
from skbio.stats.ordination import pcoa
from scipy.spatial.distance import cdist
from sklearn_extra.cluster import KMedoids
from sklearn.neural_network import MLPClassifier
from livelossplot.keras import PlotLossesCallback
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import AgglomerativeClustering
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import FunctionTransformer
from sklearn.metrics import v_measure_score, silhouette_score


In [0]:
def score_knn(k, x_train, y_train, x_test, y_test, mtc='minkowski'):
  print("score_knn")
  clf = KNeighborsClassifier(k, metric=mtc).fit(x_train, y_train)
  score = clf.score(x_test, y_test)
  pred = clf.predict(x_test)
  print('score = ', score,'v_score = ', v_measure_score(y_test, pred))

def score_k_medoids(n_cluster, x_test, y_test, mtc='minkowski'):
  print("score_k_medoids")
  clf = KMedoids(n_clusters=n_cluster, init='k-medoids++',  metric=mtc)
  pred = clf.fit_predict(x_test)
  print('v_score = ', v_measure_score(y_test, pred))

def generate_pca(x_train, y_train):
  print("generate_pca")
  pca_new = PCA(n_components=2)
  pca_new = pca.fit_transform(x_train)
  print('silhouette_score = ',silhouette_score(pca_new,y_train))

  # PCA GRAPH
  plt.scatter(pca_new[:,0],pca_new[:,1],c=y_train)
  plt.xlabel('PC1')
  plt.ylabel('PC2')
  plt.plot()

  # histogram of pca1 in relation to the number of elements for each values
  # hist, bin_edges = np.histogram(pca_new, density=True, normed=True)
  # plt.hist(pca_new, bins='auto')
  # plt.ylabel('number of elements')
  # plt.xlabel('PC1')
  # plt.title('Histogrram of PCA applied to my similarity space')
  # plt.show()

def isomap_score(x_test, y_test, n_neighs=10, n_comps=2):
  print("isomap_score")
  clf = Isomap(n_neighbors = n_neighs, n_components = n_comps)
  pred = clf.fit_transform(x_test)
  print('silhouette_score = ',silhouette_score(pred,y_test))
  plt.scatter(pred[:, 0], pred[:, 1], c=y_test)
  plt.plot()

def binary_partition_score(n_cluster, x_train, y_train, x_test, y_test, afty='euclidean'):
  print("binary_partition_score")
  clf = AgglomerativeClustering(n_cluster, linkage='average', affinity=afty)
  pred = clf.fit_predict(x_train)
  print('v_score =',v_measure_score(pred, y_train))
  print('silhouette_score = ',silhouette_score(x_test, y_test))

def pcoa_score(x_test, y_test, mtc='optional'):
  print("binary_partition_score")
  if mtc == 'optional':
    dist = cdist(x_test, x_test)
  else:
    dist = cdist(x_test, x_test, metric=mtc)
  res = pcoa(dist, number_of_dimensions=2)
  pcoa_new = res.samples.to_numpy()
  print('silhouette_score = ',silhouette_score(pcoa_new, y_test))
  plt.scatter(pcoa_new[:, 0], pcoa_new[:, 1], c=y_test)
  plt.plot()

In [0]:
# Loading and reshaping mnist data
(x_train_original, y_train), (x_test_original, y_test) = mnist.load_data()

x_train = np.reshape(x_train_original,(x_train_original.shape[0],28*28))
x_test = np.reshape(x_test_original,(x_test_original.shape[0],28*28))

# Distance Euclidienne

In [0]:
print("Distance Euclidienne")

# KNN
score_knn(6, x_train, y_train, x_test[:2000], y_test[:2000])

# K medoids
score_k_medoids(10, x_train[:20000], y_train[:20000])

# PCA
generate_pca(x_train, y_train)

# Isomap
isomap_score(x_train[:5000], y_train[:5000])

# Binary Partition
binary_partition_score(10, x_train[:10000], y_train[:10000], x_test[:2000], y_test[:2000])

# PCoA
pcoa_score(x_train[:5000], y_train[:5000])


print("\n")

# Similarité cosinus

In [0]:
train_x = x_train.reshape((-1, 28 * 28))
test_x = x_test.reshape((-1, 28 * 28))

print("Similarité cosinus")

# KNN
score_knn(1, train_x, y_train, test_x, y_test, mtc='cosine')

# K medoids
score_k_medoids(40, train_x[:2000], y_train[:2000], mtc='cosine')

# Binary Partition
binary_partition_score(100, test_x[:2000], y_test[:2000], test_x[:2000], y_test[:2000], afty='cosine')

# PCoA
pcoa_score(train_x, y_train, mtc='cosine')

print("\n")

# Siamese Net

In [0]:
### Initiation Siamese Net ###

## Definition
EPOCHS = 100
BATCH_SIZE=256
HIDDEN_DIM=10

seq = iaa.Sequential([
    iaa.Affine(
        translate_percent={'x': (-0.20, 0.20), 'y':(-0.2, 0.2)}, 
        rotate=(-35, 35),
        shear=(-20, 20),
    )
])

def generator(x, y):
  while True:
    x1s = []
    x2s = []
    ys = []
    percat=int(len(x)/20)
    for i in range(10):
      same = np.where(y == i)[0]
      x1 = x[same[np.random.choice(len(same), size=percat)]]
      x1s.append(x1)

      x2 = x[same[np.random.choice(len(same), size=percat)]]
      x2s.append(x2)

      x1 = x[same[np.random.choice(len(same), size=percat)]]
      x1s.append(x1)


      other = np.where(y != i)[0]
      x2 = x[other[np.random.choice(len(other), size=percat)]]
      x2s.append(x2)

      t = np.full((2 * percat,), 0)
      t[:percat] = 1
      ys.append(t)
    x1s=np.concatenate(x1s)
    x2s=np.concatenate(x2s)
    ys=np.concatenate(ys)
    shuffle =np.arange(len(ys))
    np.random.shuffle(shuffle)
    x1s=x1s[shuffle]
    x2s=x2s[shuffle]
    ys=ys[shuffle]
    for i in range(int(len(y)/BATCH_SIZE)):
      begin= i * BATCH_SIZE
      end = (i+1) *BATCH_SIZE
      
      r1 = seq(images=x1s[begin: end]).astype(np.float32) / 256
      r2 = seq(images=x2s[begin: end]).astype(np.float32) / 256
      y1 = ys[begin: end].astype(np.float32)
      if np.any(np.isnan(r1)) or np.any(np.isnan(r2)) or np.any(np.isnan(y1)):
        print(begin, end)  
        print('found nan')
      yield [r1, r2], y1


# make transformer
inputs = Input((28, 28, 1))
x = inputs
for i in range(5):
  x = Conv2D(2**i, (3, 3), activation='relu')(x)
x = Flatten()(x)
for i in range(5):
  x = Dense(100, activation='relu')(x)
x = Dense(HIDDEN_DIM, activation='relu')(x)
transformer = Model(inputs=inputs, outputs=x)

a = Input((28, 28, 1))
b = Input((28, 28, 1))
aa = transformer(a)
bb = transformer(b)
def f(x):
  a, b = x
  res = K.sum(K.square(a - b), axis=1, keepdims=True)
  res = K.sqrt(K.maximum(res, 2 * K.epsilon()))
  print(res)
  return res
d = Lambda(f, name='distance_layer')([aa, bb])
# ar = inverse(aa)
# br = inverse(bb)

siamese = Model(inputs=[a, b], output=d)
def contrastive_loss(y_true, y_pred):
    '''Contrastive loss from Hadsell-et-al.'06
    http://yann.lecun.com/exdb/publis/pdf/hadsell-chopra-lecun-06.pdf
    '''
    margin = 1
    square_pred = K.square(y_pred)
    margin_square = K.square(K.maximum(margin - y_pred, 0))
    return K.mean(y_true * square_pred + (1 - y_true) * margin_square)/2
def accuracy(y_true, y_pred):
    '''Compute classification accuracy with a fixed threshold on distances.
    '''
    return K.mean(K.equal(y_true, K.cast(y_pred < 0.5, y_true.dtype)))
from keras.losses import mean_squared_error
siamese.compile(optimizer='adam', loss=contrastive_loss, 
                metrics={'distance_layer': accuracy})

## Entrainement modele

train_x = x_train.reshape((-1, 28, 28, 1))
test_x = x_test.reshape((-1, 28, 28, 1))
train_x, val_x, train_y, val_y = train_test_split(train_x, y_train, test_size=0.2)

train_gen = generator(train_x, train_y)
val_gen = generator(val_x, val_y)
callbacks = [
    EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True,),
]
siamese.fit_generator(train_gen, int(len(train_y)/BATCH_SIZE), epochs=EPOCHS, callbacks=callbacks, 
                      validation_data=val_gen, validation_steps=int(len(val_y)/BATCH_SIZE))

## predictions

x_train_tr = transformer.predict(train_x)
x_test_tr = transformer.predict(test_x)
x_val_tr = transformer.predict(val_x)

## performance de cette metrique sur les algorithmes de partition

print("Siamese Net")

siamese.summary()

# KNN
score_knn(10, x_train_tr, train_y, x_val_tr, val_y)

# K medoid
score_k_medoids(10, x_val_tr, val_y)

# Isomap
isomap_score(x_test_tr, y_test)

# Binary Partition
binary_partition_score(25, x_val_tr, val_y, x_test_tr, y_test)

# PCoA
pcoa_score(x_test_tr, y_test)

print("\n")

# Hash

In [0]:
### Initiation Hash ###

## Definition

default_params = {
    'n':500,
    'dist_bin_n': 50,
    'angle_bin_n': 50,
    'dist_max': 2
}

def hash(sample,
         n=default_params['n'],
         dist_bin_n=default_params['dist_bin_n'],
         angle_bin_n=default_params['angle_bin_n'],
         dist_max=default_params['dist_max']):
    x, y = np.nonzero(sample)
    non_zero = np.array(tuple(zip(x, y)), dtype=np.float)

    p1 = non_zero[np.random.choice(len(non_zero), n)]
    p2 = non_zero[np.random.choice(len(non_zero), n)]

    delta = (p1 - p2).astype(np.float)

    # normalized distance removes scale
    distances = np.linalg.norm(delta, axis=1)
    distances /= np.mean(distances)
    distances[distances >= dist_max] = dist_max

    # if delta[, 0] is zero it's vertical
    angles = np.arctan(delta[:, 1] / delta[:, 0])
    angles[delta[:, 0] == 0] = np.pi/2
    # remove rotation
    angles += -np.mean(angles) + np.pi/2

    distance_bins = np.linspace(0, dist_max, num=dist_bin_n)
    distances_digit = np.digitize(distances, distance_bins)
    distances_digit[distances_digit==dist_bin_n] = dist_bin_n - 1

    angle_bins = np.linspace(0, np.pi+0.001, num=angle_bin_n)
    angles_digit = np.digitize(angles, angle_bins)
    angles_digit[angles_digit == angle_bin_n] -= 1

    store = np.zeros((dist_bin_n, angle_bin_n), dtype=np.int)
    for x, y in zip(distances_digit, angles_digit):
        store[x, y] += 1
    store = store / np.sum(store)
    return store

def make_hash_transformer(**args):
    for i in args.keys():
        if i not in default_params:
            raise Exception('inavlid param: {}'.format(i))
    for i in default_params.keys():
        if i not in args:
            args[i] = default_params[i]

    def func(x):
        return np.c_[[hash(i, **args).reshape(-1) for i in tqdm(x)]]

    return FunctionTransformer(func, validate=False)


## transform data
tr = make_hash_transformer(angle_bin_n=10, dist_bin_n=10, dist_max=2)
x_train_hash = tr.fit_transform(x_train_original)
x_test_hash = tr.fit_transform(x_test_original)

## performance de cette metrique sur les algorithmes de partition
print("Hash")

# KNN
score_knn(100, x_train_hash, y_train, x_test_hash, y_test)

# K medoid
score_k_medoids(100, x_test_hash, y_test)

# Isomap
isomap_score(x_test_hash, y_test, n_neighs=10, n_comps=2)

# Binary Partition
binary_partition_score(20, x_train_hash, y_train, x_test_hash, y_test)

# PCoA
pcoa_score(x_test_hash, y_test)

print("\n")

# Distance avec d'autre vecteurs

In [0]:
train_x = x_train.reshape((-1, 28 * 28))
test_x = x_test.reshape((-1, 28 * 28))

print("Similarité cosinus")

n = 1000
select = np.random.choice(len(train_x), n, replace=False)
sterotype=x_train[select]
new_x_test = cdist(test_x, sterotype)
new_x_train = cdist(train_x, sterotype)

# KNN
score_knn(10, new_x_train, y_train, new_x_test, y_test)

# K medoids
score_k_medoids(40, new_x_test, y_test)

# Isomap
isomap_score(new_x_test, y_test, n_neighs=10, n_comps=2)

# Binary Partition
binary_partition_score(40, new_x_test, y_test, new_x_test, y_test)

# PCoA
pcoa_score(new_x_test, y_test)

print("\n")

# Similarite avec reseau de neurones

In [0]:
### Initiation similarite avec reseau de neurones ###

## Definition
clf = MLPClassifier(hidden_layer_sizes=(128,128,128),
                          batch_size=100,
                          #verbose=True,
                          max_iter=150,
                          n_iter_no_change=25,
                          random_state=42)
clf.fit(x_train,y_train)

# Pickle transformed data to save time
# pickle.dump(clf,open('mlp_clf.pickle','wb'))
# clf = pickle.load(open('mlp_clf.pickle','rb'))

## predictions en utilisant le reseau de neuronnes

x_train_new_rn = clf.predict_proba(x_train)
x_test_new_rn = clf.predict_proba(x_test)

## performance de cette metrique sur les algorithmes de partition

print("Similarite avec reseau de neurones")

# KNN
score_knn(6, x_train_new_rn, y_train, x_test_new_rn[:2000], y_test[:2000])

# K medoid
score_k_medoids(10, x_train_new_rn[:20000], y_train_new[:20000])

# PCA
generate_pca(x_train_new_rn, y_train_new)

# Isomap
isomap_score(x_train_new_rn[:5000], y_train[:5000])

# Binary Partition
binary_partition_score(10, x_train_new_rn[:10000], y_train[:10000], x_test_new_rn[:2000], y_test[:2000])

# PCoA
pcoa_score(x_train_new_rn[:5000], y_train[:5000])

print("\n")

## Invariance aux modifications

In [0]:
def similarity(x_1,x_2) :
    probas_x_1,probas_x_2 = clf.predict_proba([x_1, x_2])
    #print(probas_x_1,probas_x_2)
    return np.linalg.norm(probas_x_1-probas_x_2)

def show_image (x,y):
    plt.imshow(x.reshape((28,28)))
    plt.title(str(y))
    plt.show()

seq = iaa.Sequential([
    iaa.Affine(
        translate_percent={'x': (-0.10, 0.10), 'y':(-0.1, 0.1)}, 
        rotate=(-35, 35),
        shear=(-10, 10),
    )
])

rts = seq(images=x_train[:25].reshape((x_train[:25].shape[0],28,28)))
rts = rts.reshape((rts.shape[0],28*28))

show_image(rts[7],'3 with modifications')
show_image(x_train[7],'original 3')
show_image(x_train[12],'other 3')
show_image(x_train[17],'an original 8')

print("sim entre 3 mod et 3 original :",similarity(rts[7],x_train[7]))
print("sim entre 3 original et autre 3 :",similarity(x_train[7],x_train[12]))
print("sim entre 3 mod et autre 3 :",similarity(rts[7],x_train[12]))
print("sim entre 3 mod et 8 : ",similarity(rts[7],x_train[17]))
print("sim entre 3 original et 8 :",similarity(x_train[7],x_train[17]))
print("sim entre autre 3 et 8 :",similarity(x_train[12],x_train[17]))